In [3]:
import os
import warnings
import copy
import re

import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date


PATH = '/content/'

In [1]:
!mkdir final_tables
!mkdir out_tables

# НОВОЕ перерабоьтанное   
- ищет дату в датафрейме
- берет значение справа с текстом

In [51]:
from tables.utils import string_to_classes
######### all functions

df_result = pd.DataFrame( columns = ['Дата донации', 'Класс крови' , 'Тип донации' ])

# очистить потенциальную ячейку дату от возможных косяков
def clear_data(string):
    date_string = string
    if len(date_string) >= 6:
        date_string = string.replace(',','.').replace(' ','.').replace(',','.')
    return date_string

# попытка конверт в дату значения ячейки, если ошибка , то возращает значние
def convert_to_date(string):
    date_string = string
    #для дат формата xx.xx.xxxx
    if len(re.sub(r'\D', '', string)) >= 7:
      try:
          return datetime.strptime(date_string, '%d.%m.%Y').date()
      except ValueError:
          return string
    # для дат формата xx.xx.xx
    elif   len(re.sub(r'\D', '', string)) <= 6:
      try:
          return datetime.strptime(date_string, '%d.%m.%y').date()
      except ValueError:
          return string
    #
    else:
      return string

# Определение функции для проверки наличия значения типа date в строке
# добавление последущего значени я в строке
def find_date_in_dataframe(row):
    datetime_values = []
    for i in range(len(row)):
        row_dict = {}
        if isinstance(row[i], date):
            datetime_values.append([row[i] ,row[i+1]])
    return datetime_values

# перевод списка списков из даты и текста после поиска в датафреме в дата фрейм
def transform_list_to_df(df):
    # Применение функции поиска даты к каждой строке датасета
    result = df.apply(find_date_in_dataframe, axis=1)

    # превращение списка списков в датефрейм
    columns = {'Дата донации': [], 'text': [] }
    for sublist in result:
        for item in sublist:
            columns['Дата донации'].append(item[0])
            columns['text'].append(item[1])
    # Создание датафрейма из словаря
    df_out = pd.DataFrame(columns)
    return df_out


#type of blood
def find_type_blood(string):
    plazma = {'пл', 'л', 'п' }
    blood = {'кр', 'р', 'к'}
    trom = {'т', 'ф'}
    substrings = [*re.sub('[^а-яА-Я]', '', string)]
    if len(set(substrings) & plazma) > 0:
        string = 'Плазма'
    elif len(set(substrings) & blood) > 0:
        string = 'Цельная кровь'
    elif len(set(substrings) & trom ) > 0:
        string = 'Тромбоциты'
    else:
        string = '-'
    return string

# Безвомездно или Платно
def find_free_donat(string):
      free = {'б','в' , 'бв', '6', '9', '69','66','0'}
      not_free = {'плат', 'лат', 'пла' , 'ат'}
      #substrings = [*re.sub('[^а-яА-Я]', '', string)]
      substrings = [*string]
      if len(set(substrings) & free) > 0:
          string = 'Безвозмездно'
      elif len(set(substrings) & not_free) > 0:
          string = 'Платно'
      else:
        string = '-'
      return string


def processing_dataset_after_OCR(filename):
    # 1 для каждого столбца пройтись по ячейкам и пробовать преобразовать в дату
    # 2 затем датфрейм получаем с датой+текст
    # 3 затем тект раскладываем на две колонки, а дату в стринг нужного формата

    #0 load dataset
    df = pd.read_csv(filename)
    #1
    columns_list = df.columns.to_list()
    for column in columns_list:
      df[column] = df[column].astype('str')
      df[column] = df[column].map(clear_data)
      df[column] = df[column].map(convert_to_date)
    #2
    df_out = transform_list_to_df(df)

    #3
    df_out['Класс крови'] = df_out['text'].map(find_type_blood)
    df_out['Тип донации'] = df_out['text'].map(find_free_donat)
    df_out = df_out.sort_values(by='Дата донации',ascending=True)
    df_out['Дата донации'] =  df_out['Дата донации'].map(lambda x: x.strftime('%d.%m.%Y'))
    df_out = df_out.reset_index(drop=True)

    return  df_out


In [56]:
filename = '/content/out_tables/227414.csv'
#filename = '/content/out_tables/225629 .csv'
df = pd.read_csv(filename)
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,-,-,-,-,-,-,(ПОДПиСь),-,-,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-,-,-,-,Отметки 0 взятии крови (плазмы),-,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Дата,Вид,Кол-во,Дата,Вид,Кол-во,Дата,Вид,Кол-во,NaN,NaN,NaN,NaN,NaN,NaN
3,3,-,дон-ва,-,-,дон-ва,-,-,дон-ва,-,NaN,NaN,NaN,NaN,NaN,NaN
4,4,22.08.19,2,к/д(бв),2,к/д(бв),3,22.10.19,5,т/ф(бв),5.0,т/ф(бв),-,-,8.0,9.0
5,5,-,-,450,-,-,525,21.11.19,т/ф(бв),442,NaN,NaN,NaN,NaN,NaN,NaN
6,6,29.05.20,т/ф(бв),430,10.07.20,таф(бв),430,06.08.20,т/ф(бв),345,NaN,NaN,NaN,NaN,NaN,NaN
7,7,25.08.20,тАф(бв),435,11.03.21,пф(бв),600,07.04.21,пф(бв),600,NaN,NaN,NaN,NaN,NaN,NaN
8,8,22.04.21,пф(бв),600,31.08.21,таф(бв),356,22.09.21,т/ф(бв),345,NaN,NaN,NaN,NaN,NaN,NaN
9,9,29.12.21,т/ф(бв),500,08.02.22,таф(бв),342,"23.03,22",т/ф(бв),342,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
processing_dataset_after_OCR(filename) #.drop('text', axis=1)

#если не нужна колонка с исходным тексто то добавить .drop('text', axis=1)

,Дата донации,text,Класс крови,Тип донации
0,22.08.2019,2,-,-
1,22.10.2019,5,-,-
2,21.11.2019,т/ф(бв),Тромбоциты,Безвозмездно
3,29.05.2020,т/ф(бв),Тромбоциты,Безвозмездно
4,10.07.2020,таф(бв),Тромбоциты,Безвозмездно
5,06.08.2020,т/ф(бв),Тромбоциты,Безвозмездно
6,25.08.2020,тАф(бв),Тромбоциты,Безвозмездно
7,11.03.2021,пф(бв),Плазма,Безвозмездно
8,07.04.2021,пф(бв),Плазма,Безвозмездно
9,22.04.2021,пф(бв),Плазма,Безвозмездно


# Перевод всех таблиц

In [58]:
PATH = '/content/out_tables/'
PATH_final = '/content/final_tables/'
# Перебор всех файлов в папке
for file_name in os.listdir(PATH):
    if file_name.endswith('.csv'):
        # Полный путь к файлу
        file_path = os.path.join(PATH, file_name)
        # Вызов функции для каждого файла CSV
        print (file_name)
        print(file_path)

        df_out = processing_dataset_after_OCR(file_path).drop('text', axis=1)

        df_out.to_csv(f'{PATH_final}final{file_name}')


225629 .csv
/content/out_tables/225629 .csv
228963 .csv
/content/out_tables/228963 .csv
204119 .csv
/content/out_tables/204119 .csv
141899 .csv
/content/out_tables/141899 .csv
231820 .csv
/content/out_tables/231820 .csv
256578 .csv
/content/out_tables/256578 .csv
256838.csv
/content/out_tables/256838.csv
254586 .csv
/content/out_tables/254586 .csv
213950.csv
/content/out_tables/213950.csv
227414.csv
/content/out_tables/227414.csv
243478 .csv
/content/out_tables/243478 .csv
236000 .csv
/content/out_tables/236000 .csv
245365 .csv
/content/out_tables/245365 .csv
238716.csv
/content/out_tables/238716.csv
233749 .csv
/content/out_tables/233749 .csv
240493.csv
/content/out_tables/240493.csv


In [50]:
string = '02.05.22'
convert_to_date(string)

datetime.date(2022, 5, 2)